In [18]:
# in this we are going to use logistic regression model as this is a binary classification project
#binary classification means we are going to classify the result into two types real or fake for this logis is best
#what all are there in the dataset
#1.id:unique id for a news article
#2.title:the title of a news article
#3.author:author of the news article
#4.text:the text of the article,could be incomplete
#5.label:a label that marks the news fake or real



In [19]:
#0:real news
#1:fake news

In [1]:
import numpy as np
import pandas as pd
# regular expression(re) is used for searching words in the text in the document
import re
from nltk.corpus import stopwords
#stopword means removing unuseful data like a,and,the
#corpus means body of particular text(important content)
#nltk stands for natural language tool kit
from nltk.stem.porter import PorterStemmer
#stemmer takes a word and removes prefix and suffix and returns root word
from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer is used to convert text into feature vectors(numbers)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [20]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Data pre-processing

In [25]:
news=pd.read_csv('/content/train.csv')

In [26]:
news

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [21]:
#20,800 rows and 5 columns
#around 20800 news articles 5 features
news.shape

(20800, 6)

In [22]:
news.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...


In [23]:
##counting the no.of misssing values in dataset
news.isnull().sum()

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

In [6]:
##replacing null values with empty string
news=news.fillna('')

In [7]:
#we are going to use title and author because text column is too large and takes more time for processing
#merging author name news title
news['content']=news['author']+' '+news['title']

In [24]:
news

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,jerom hudson rapper trump poster child white s...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,benjamin hoffman n f l playoff schedul matchup...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,michael j de la merc rachel abram maci said re...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,alex ansari nato russia hold parallel exercis ...


In [8]:
print(news['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


# **Stemming**
ex:
player,playing : root word is play
removes prefix and suffix

In [9]:
portstem = PorterStemmer()

In [10]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  #this particular line removes all the content such as number and other symbols other than alphabetes and will replaced with content
  #and those numbers will be replaced with spaces
  stemmed_content = stemmed_content.lower()
  # converting all the letters to lower case letters otherwise model may think those words as significant so to remove those complexities
  stemmed_content = stemmed_content.split()
  # splitting the dataset converted to list
  stemmed_content = [portstem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #printing the words without stop words with the help of for loop by removing prefix and suffix
  stemmed_content = ' '.join(stemmed_content)
  #after that joining all functions
  return stemmed_content

In [11]:
news['content'] = news['content'].apply(stemming)

In [12]:
print(news['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [13]:
#seperating the data and the label
X=news['content'].values
Y=news['label'].values

In [14]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [15]:
print(Y)

[1 0 1 ... 0 1 1]


In [16]:
Y.shape

(20800,)

In [17]:
#converting textual data into numerical
#term frequency inverse document frequency counts number of times a particular word is repeating in a document in a particular text or paragraph
#repetition tells the model that it is very important and it assigns a particular numerical value that is the purpose of this tf
#idf removes the significance for the useless texts which are repeating more times by this they create feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)
#fits and tranforms to numbers



In [27]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

# Splitting the text into train and test

In [29]:
#variable Y contains 01 if we dont mention stratify real news and fake news will not be seggregated in equal proportion
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

# Training the model

In logistic regression model S represents sigmoid
y=1/1+e^-z
when we plot the values in the equation we get the curve
there will be a threshold value 
based on that model will decide whether it is fake or real news
Z=w.X+b
X-input features(content)
Y-prediction probability(0 or 1)
w-weights(represents how important the feature is like author column will have more weight 
b-biases intercept


In [30]:
model=LogisticRegression()

In [31]:
#fits the data to the sigmoid curve
model.fit(X_train,Y_train)

LogisticRegression()

# Evaluation

In [33]:
#accuarcy score on training data
X_train_prediction=model.predict(X_train)
#
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [34]:
print('Accuracy score of the training data',training_data_accuracy)

Accuracy score of the training data 0.9865985576923076


In [35]:
#accuarcy score on training data
X_test_prediction=model.predict(X_test)
#
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [36]:
print('Accuracy score of testing data',test_data_accuracy)

Accuracy score of testing data 0.9790865384615385


# Making a predictive system

In [44]:
X_new=X_test[1]
prediction=model.predict(X_new)
print(prediction)
if prediction[0]==0:
  print('Its a real 🗞' )
else:
  print('Its a fake news 📰 ')

[0]
Its a real 🗞
